In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import backend as K
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,EarlyStopping
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [10]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

def drop_vars(df):
    tmp=df.shape[1]
    df = df[df.columns[[True]+list((df.var()!=0))]]
    print('0 var:',tmp-df.shape[1])
    
    corr_matrix = df[df.columns[2:]].corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
    tmp=df.shape[1]
    df=df.drop(to_drop, axis=1)
    print('Corr>0.95:',tmp-df.shape[1],'Now:',)
    
    corrs = dict()
    for i in range(df.shape[1]-2):
        corrs[df.columns[2+i]] = np.corrcoef(df['target'],df[df.columns[2+i]])[0,1]
    s = [k for k in corrs if abs(corrs[k])<0.1]
    tmp=df.shape[1]
    df=df.drop(s, axis=1)
    print('Corr Target <0.1:',tmp-df.shape[1])
    
    return df
train = drop_vars(train)

0 var: 256
Corr>0.95: 139 Now:
Corr Target <0.1: 4206


## Test Improvement

In [21]:
X=train[train.columns[2:]]
y=np.log1p(train['target'])

x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=203)

sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 

clf = ElasticNet(random_state=23,alpha=5)
clf.fit(x_train, y_train)

y_true,y_pred = y_test,clf.predict(x_test) 
print(root_mean_squared_error(y_pred, y_true))

scal=StandardScaler()
x_train = np.c_[x_train,scal.fit_transform(clf.predict(x_train).flatten().reshape(-1,1)).flatten()]
x_test = np.c_[x_test,scal.transform(clf.predict(x_test).flatten().reshape(-1,1)).flatten()]

model = Sequential()
model.add(Dense(50, input_dim=x_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1))

model.compile(loss=root_mean_squared_error,
              optimizer=Adam(lr=0.1,decay=0.0001))


checkp = ModelCheckpoint(filepath='weights.hdf5')
lrred = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, cooldown=2, min_lr=0.000001)
stp = EarlyStopping(monitor='val_loss', min_delta=0, patience=50)
cbs = [checkp,lrred,stp]
model.fit(x_train, y_train,
        epochs=1000,
        batch_size=400,
        validation_data=(x_test, y_test),
        callbacks=cbs)

Tensor("Sqrt_5:0", shape=(), dtype=float64)
Train on 3567 samples, validate on 892 samples
Epoch 1/1000
3567/3567 [==============================] - 0s - loss: 16.2245 - val_loss: 8.1600
Epoch 2/1000
3567/3567 [==============================] - 0s - loss: 10.5507 - val_loss: 8.6074
Epoch 3/1000
3567/3567 [==============================] - 0s - loss: 8.6955 - val_loss: 6.3128
Epoch 4/1000
3567/3567 [==============================] - 0s - loss: 7.7951 - val_loss: 5.3575
Epoch 5/1000
3567/3567 [==============================] - 0s - loss: 6.7558 - val_loss: 5.8245
Epoch 6/1000
3567/3567 [==============================] - 0s - loss: 6.3329 - val_loss: 5.2231
Epoch 7/1000
3567

## NN+ElNet - Actual

In [19]:
x_train=train[train.columns[2:]]
y_train=np.log1p(train['target'])

x_test=test[x_train.columns]

sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 

clf = ElasticNet(random_state=23,alpha=4)
clf.fit(x_train, y_train)

scal=StandardScaler()
x_train = np.c_[x_train,scal.fit_transform(clf.predict(x_train).flatten().reshape(-1,1)).flatten()]
x_test = np.c_[x_test,scal.transform(clf.predict(x_test).flatten().reshape(-1,1)).flatten()]

model = Sequential()
model.add(Dense(50, input_dim=x_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1))

model.compile(loss=root_mean_squared_error,
              optimizer=Adam(lr=0.1,decay=0.0001))


checkp = ModelCheckpoint(filepath='weights.hdf5')
lrred = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=10, cooldown=2, min_lr=0.000001)
stp = EarlyStopping(monitor='loss', min_delta=0, patience=50)
cbs = [checkp,lrred,stp]
model.fit(x_train, y_train,
        epochs=1000,
        batch_size=400,
        callbacks=cbs)

Epoch 1/1000
4459/4459 [==============================] - 0s - loss: 13.8220     
Epoch 2/1000
4459/4459 [==============================] - 0s - loss: 8.8388     
Epoch 3/1000
4459/4459 [==============================] - 0s - loss: 7.5953     
Epoch 4/1000
4459/4459 [==============================] - 0s - loss: 6.5133     
Epoch 5/1000
4459/4459 [==============================] - 0s - loss: 6.0867     
Epoch 6/1000
4459/4459 [==============================] - 0s - loss: 5.2933     
Epoch 7/1000
4459/4459 [==============================] - 0s - loss: 4.7450     

In [20]:
predictions=pd.DataFrame({'ID':test['ID'],'target':np.expm1(model.predict(x_test, verbose=1).flatten())})
print(predictions.head())
predictions.to_csv('pred_stack.csv',index=False)

47456/49342 [===========================>..] - ETA: 0s 